In [294]:
import pandas as pd
import psycopg2 as pg2
import numpy as np

In [295]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

Connecting to SQL


In [296]:
print ("Reading Dataset")
df_giveups = pd.read_sql ("""
Select username, count(*) as giveups, avg(perc_to_bin) as avg_perc_to_bin, avg(bids_so_far) as g__avg_bid, avg(bom_bids_so_far) as g_bom_bid_avg, avg(bom_streak) as g_bom_bids_avg from auction_full where giveup group by username
""", conn, index_col="username")
df_giveups

Reading Dataset


,giveups,avg_perc_to_bin,g__avg_bid,g_bom_bid_avg,g_bom_bids_avg
username,,,,,
001167,3,0.385650,47.000000,44.666667,20.333333
007dealbreaker,1,0.006452,1.000000,0.000000,0.000000
03fatboy,38,0.062715,3.842105,0.000000,0.000000
0415Naruto,4,0.489450,38.250000,14.750000,0.000000
0621bev,8,0.274806,31.625000,4.000000,2.750000
...,...,...,...,...,...
zstryjewski,3,0.065983,9.000000,0.000000,0.000000
zuber1992,1,0.825806,128.000000,35.000000,0.000000
zwetzig,13,0.059543,6.923077,0.000000,0.000000


In [297]:
df_auctions = pd.read_sql(""" 
SELECT username, count(*) as auctions, avg(bid) as avg_debut, avg(cashvalue) as avg_cashvalue 
from auction_full where debut=1 group by username having count(*) > 10
""", conn, index_col="username")
df_auctions.fillna(0, inplace=True)

In [298]:
print ("Reading Dataset")
df_wins = pd.read_sql ("""
Select username, avg(is_winner::int) as win_perc, avg(cashvalue - fee - bid/100-.4) as proft_avg from auction_full  group by username
"""
#, stddev(bids_so_far) as w_bid_var, avg(bids_so_far) as w_bid_avg, avg(bom_bids_so_far) as w_bom_bids_avg from auction_full where is_winner group by username
, conn, index_col="username")
df_wins

Reading Dataset


,win_perc
username,
001167,0.0
007dealbreaker,0.0
03fatboy,0.0
0415Naruto,0.0
0621bev,0.0
...,...
zstryjewski,0.0
zuber1992,0.0
zwetzig,0.0


In [299]:
df_auctions

,auctions,avg_debut,avg_cashvalue
username,,,
03fatboy,38,67.500000,45.078947
07999,47,89.340426,55.021277
07robert,2592,82.984182,43.800540
0988dandu,38,164.657895,68.947368
10000000000Bids,89,8.011236,36.606742
...,...,...,...
zoeybelleziz,17,80.823529,44.882353
zombiemomma,107,43.308411,38.373832
zoomer2u,127,62.889764,51.448819


In [300]:
df = df_auctions.merge(df_giveups, how="left", left_on="username", right_on="username")

In [301]:
df = df.merge(df_wins, left_on="username", right_on="username", how="left")

In [302]:
X=df
X.pop("auctions")
X.pop("giveups")

username
03fatboy             38
07999                35
07robert           2393
0988dandu            35
10000000000Bids      87
                   ... 
zoeybelleziz         16
zombiemomma          95
zoomer2u            105
zwetzig              13
zzrats               12
Name: giveups, Length: 2689, dtype: int64

In [303]:
win_perc = X.pop("win_perc")

In [304]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler



In [305]:
#ss = StandardScaler()
#X = ss.fit_transform(X)

In [306]:
km = KMeans(n_clusters=5)
km.fit(X)
#from sklearn.decomposition import NMF
#nmf = NMF(5)
#W = nmf.fit_transform(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [307]:
#H = nmf.components_

In [308]:
groups = pd.Series(km.predict(X))

In [309]:
final = pd.concat([df.reset_index(),win_perc.reset_index(), groups], axis=1)


In [310]:
final

,username,avg_debut,avg_cashvalue,avg_perc_to_bin,g__avg_bid,g_bom_bid_avg,g_bom_bids_avg,username,win_perc,0
0,03fatboy,67.500000,45.078947,0.062715,3.842105,0.000000,0.000000,03fatboy,0.000000,1
1,07999,89.340426,55.021277,0.411417,49.057143,42.028571,13.514286,07999,0.005714,0
2,07robert,82.984182,43.800540,0.211891,19.561638,14.708316,6.210196,07robert,0.003927,1
3,0988dandu,164.657895,68.947368,0.093690,12.800000,10.600000,10.457143,0988dandu,0.005484,0
4,10000000000Bids,8.011236,36.606742,0.087135,5.954023,2.022989,0.620690,10000000000Bids,0.003559,1
...,...,...,...,...,...,...,...,...,...,...
2684,zoeybelleziz,80.823529,44.882353,0.151327,23.125000,20.562500,12.062500,zoeybelleziz,0.002695,1
2685,zombiemomma,43.308411,38.373832,0.194079,11.010526,6.621053,1.978947,zombiemomma,0.007727,1
2686,zoomer2u,62.889764,51.448819,0.695327,89.371429,88.942857,86.952381,zoomer2u,0.002108,3
2687,zwetzig,20.153846,56.000000,0.059543,6.923077,0.000000,0.000000,zwetzig,0.000000,1


In [311]:
final.groupby(0).mean()

,avg_debut,avg_cashvalue,avg_perc_to_bin,g__avg_bid,g_bom_bid_avg,g_bom_bids_avg,win_perc
0,,,,,,,
0,129.223397,50.910814,0.149118,15.996162,9.499220,5.707555,0.002543
1,44.408279,41.658470,0.203679,17.816293,11.747168,7.450859,0.004263
2,368.586034,71.271177,0.166163,27.199622,20.076046,12.271870,0.002646
3,105.839931,56.420419,0.692927,96.118425,88.477865,67.680477,0.002465
4,228.739908,59.632337,0.148670,19.532516,12.904979,8.786731,0.002329


In [174]:
df_wins.wins

username
07999               12
07robert           199
0988dandu            3
10000000000Bids      2
1000sOFBids          1
                  ... 
zeplin               5
zilda123           102
zoeybelleziz         1
zombiemomma         12
zoomer2u            22
Name: wins, Length: 1992, dtype: int64

In [312]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
df = pd.read_sql ("""Select *  from auction_full where auctiontime >= '2020-01-01' LIMIT 1""", conn)


Connecting to SQL
Reading Dataset
